In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/BPI2016_Werkmap_Messages.csv', delimiter=';')

In [ ]:
df.head()

,CustomerID,AgeCategory,Gender,Office_U,Office_W,EventDateTime,EventType,HandlingChannelID
0,2032131,18-29,M,271,271,2015-11-02 01:36:53.0000000,Werkmap message,1
1,2032131,18-29,M,271,271,2015-11-05 21:35:43.0000000,Werkmap message,1
2,2032131,18-29,M,271,271,2015-11-06 15:47:15.0000000,Werkmap message,1
3,2085395,18-29,V,280,280,2015-10-20 23:44:39.0000000,Werkmap message,2
4,2085395,18-29,V,280,280,2016-01-20 23:04:25.0000000,Werkmap message,2


In [ ]:
import random

# Convert EventDateTime to datetime type
df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])

# Group by EventDateTime and filter groups with more than one row
similar_event_datetimes = df.groupby('EventDateTime').filter(lambda x: len(x) > 1)

# Randomly select one group
selected_group = random.choice(similar_event_datetimes['EventDateTime'].unique())

# Filter the DataFrame to show only rows from the randomly selected group
random_group_rows = df[df['EventDateTime'] == selected_group]

print(random_group_rows)

      CustomerID AgeCategory Gender  Office_U  Office_W       EventDateTime  \
5067     1963461       50-65      V       327       327 2016-01-06 16:39:37   
6455      575494       30-39      M       280       285 2016-01-06 16:39:37   

            EventType  HandlingChannelID  
5067  Werkmap message                  1  
6455  Werkmap message                  1  


HOSPITAL LOG

In [2]:
!pip install pm4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26096 sha256=ec1aad185fc60edc1102a8021b89d5341747c5238c3075df4927bd76d50418d2
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


In [3]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer

event_log = xes_importer.apply('/content/drive/My Drive/form1.xes')
# event_log = xes_importer.apply('form1.xes')

/usr/local/lib/python3.10/dist-packages/pm4py/util/dt_parsing/parser.py:77: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
# Display the first few rows of the event log
print(event_log[:5])

[{'attributes': {'concept:name': 'case_824'}, 'events': [{'concept:name': 'Activity A', 'time:timestamp': datetime.datetime(1970, 1, 1, 5, 30, tzinfo=datetime.timezone.utc)}, '..', {'concept:name': 'Activity L', 'time:timestamp': datetime.datetime(1970, 1, 1, 10, 30, tzinfo=datetime.timezone.utc)}]}, {'attributes': {'concept:name': 'case_907'}, 'events': [{'concept:name': 'Activity A', 'time:timestamp': datetime.datetime(1970, 1, 1, 5, 30, tzinfo=datetime.timezone.utc)}, '..', {'concept:name': 'Activity L', 'time:timestamp': datetime.datetime(1970, 1, 1, 10, 30, tzinfo=datetime.timezone.utc)}]}, {'attributes': {'concept:name': 'case_659'}, 'events': [{'concept:name': 'Activity A', 'time:timestamp': datetime.datetime(1970, 1, 1, 5, 30, tzinfo=datetime.timezone.utc)}, '..', {'concept:name': 'Activity L', 'time:timestamp': datetime.datetime(1970, 1, 1, 10, 30, tzinfo=datetime.timezone.utc)}]}, {'attributes': {'concept:name': 'case_742'}, 'events': [{'concept:name': 'Activity A', 'time:tim

In [ ]:
flatten_data = []

# Iterate over all events in the event log
for trace in event_log:
    for event in trace:
        # Extract attributes and event data
        attributes = {key: event.attributes[key] for key in event.attributes}
        event_data = {key: event[key] for key in event}

        # Combine attributes and event data into a single dictionary
        row = {**attributes, **event_data}

        # Append the dictionary to the list
        flatten_data.append(row)

# Create DataFrame from the list of dictionaries
df = pd.DataFrame(flatten_data)

# Display the first few rows of the DataFrame
print(df.head())

AttributeError: 'Event' object has no attribute 'attributes'

In [ ]:
flatten_data = []

# Iterate over each case and its events
for case in event_log:
    case_id = case['attributes']['concept:name']
    for event in case['events']:
        event_data = {
            'CaseID': case_id,
            'Activity': event['concept:name'],
            'Timestamp': pd.Timestamp(event['time:timestamp'])
        }
        flatten_data.append(event_data)

# Create DataFrame from the list of dictionaries
df = pd.DataFrame(flatten_data)

# Display the structured DataFrame
print(df)

NameError: name 'data' is not defined

In [6]:
import pandas as pd
flatten_data = []

# Iterate over all traces in the event log
for trace in event_log:
    # Iterate over all events in the trace
    for event in trace:
        # Extract event data
        event_data = {key: event[key] for key in event}

        # Append the event data to the list
        flatten_data.append(event_data)

# Create DataFrame from the list of dictionaries
df = pd.DataFrame(flatten_data)

# Display the first few rows of the DataFrame
# print(df.head())
df.head()

,concept:name,time:timestamp,lifecycle:transition,variable_a
0,Activity A,1970-01-01 05:30:00+00:00,NaN,NaN
1,Activity D,1970-01-01 06:30:00+00:00,start,NaN
2,Activity E,1970-01-01 06:30:00+00:00,NaN,NaN
3,Activity F,1970-01-01 06:30:00+00:00,NaN,NaN
4,Activity C,1970-01-01 06:30:00+00:00,NaN,NaN


In [11]:
import xml.etree.ElementTree as ET
from datetime import datetime
import random

# XML data
# xes_data = '''
# <!-- your XES data here -->
# '''

xes_file_path = "form3.xes"

# Parse XML
tree = ET.parse(xes_file_path)
root = tree.getroot()

# Parse XML
# root = ET.fromstring(xes_data)

# Extract data from XML elements
cases = {}
for trace in root.findall('.//trace'):
    case_id = trace.find('.//string[@key="concept:name"]').attrib['value']
    events = []
    for event in trace.findall('.//event'):
        event_name = event.find('.//string[@key="concept:name"]').attrib['value']
        event_timestamp = event.find('.//date[@key="time:timestamp"]').attrib['value']
        events.append((event_name, event_timestamp))
    cases[case_id] = events

# Function to check if timestamps are similar
def is_similar_timestamp(timestamp1, timestamp2, threshold_minutes=5):
    time_format = '%Y-%m-%dT%H:%M:%S%z'
    dt1 = datetime.strptime(timestamp1, time_format)
    dt2 = datetime.strptime(timestamp2, time_format)
    time_difference = abs((dt1 - dt2).total_seconds()) / 60  # in minutes
    return time_difference <= threshold_minutes

# Find groups with similar timestamps
similar_groups = []
for case_id, events in cases.items():
    for other_case_id, other_events in cases.items():
        if case_id == other_case_id:
            continue
        for event, timestamp in events:
            for other_event, other_timestamp in other_events:
                if is_similar_timestamp(timestamp, other_timestamp):
                    similar_groups.append((case_id, other_case_id))
                    break

# Randomly select a group
selected_group = random.choice(similar_groups)
print("Randomly selected group with similar timestamps:")
print(selected_group)

# Print events for the selected group
for case_id, events in cases.items():
    if case_id in selected_group:
        print(f"Case ID: {case_id}")
        print("Events:")
        for event_name, event_timestamp in events:
            print(f"\tEvent: {event_name}, Timestamp: {event_timestamp}")


Randomly selected group with similar timestamps:
('case_8', 'case_10')
Case ID: case_10
Events:
	Event: Activity A, Timestamp: 1970-01-01T05:30:00+05:30
	Event: Activity C, Timestamp: 1970-01-01T06:30:00+05:30
	Event: Activity D, Timestamp: 1970-01-01T06:30:00+05:30
	Event: Activity F, Timestamp: 1970-01-01T06:30:00+05:30
	Event: Activity E, Timestamp: 1970-01-01T06:30:00+05:30
	Event: Activity B, Timestamp: 1970-01-01T07:30:00+05:30
	Event: Activity G, Timestamp: 1970-01-01T08:30:00+05:30
	Event: Activity H, Timestamp: 1970-01-01T08:30:00+05:30
	Event: Activity I, Timestamp: 1970-01-01T08:30:00+05:30
	Event: Activity J, Timestamp: 1970-01-01T08:30:00+05:30
	Event: Activity K, Timestamp: 1970-01-01T09:30:00+05:30
	Event: Activity N, Timestamp: 1970-01-01T10:30:00+05:30
	Event: Activity L, Timestamp: 1970-01-01T11:30:00+05:30
Case ID: case_8
Events:
	Event: Activity A, Timestamp: 1970-01-01T05:30:00+05:30
	Event: Activity C, Timestamp: 1970-01-01T06:30:00+05:30
	Event: Activity E, Times

In [22]:
import xml.etree.ElementTree as ET
from datetime import datetime

# Path to the XES file
xes_file_path = "form3.xes"

# Parse XML
tree = ET.parse(xes_file_path)
root = tree.getroot()

# Extract data from XML elements
cases = {}
for trace in root.findall('.//trace'):
    case_id = trace.find('.//string[@key="concept:name"]').attrib['value']
    events = []
    for event in trace.findall('.//event'):
        event_name = event.find('.//string[@key="concept:name"]').attrib['value']
        event_timestamp = event.find('.//date[@key="time:timestamp"]').attrib['value']
        events.append((event_name, event_timestamp))
    cases.setdefault(case_id, []).extend(events)

# Function to check if timestamps are similar
def is_similar_timestamp(timestamp1, timestamp2, threshold_minutes=5):
    time_format = '%Y-%m-%dT%H:%M:%S%z'
    dt1 = datetime.strptime(timestamp1, time_format)
    dt2 = datetime.strptime(timestamp2, time_format)
    time_difference = abs((dt1 - dt2).total_seconds()) / 60  # in minutes
    return time_difference <= threshold_minutes

# Find groups with similar timestamps and case IDs
similar_groups = {}
for case_id1, events1 in cases.items():
    for case_id2, events2 in cases.items():
        if case_id1 == case_id2:
            # continue
          for event1 in events1:
              for event2 in events2:
                  event_name1, timestamp1 = event1
                  event_name2, timestamp2 = event2
                  if is_similar_timestamp(timestamp1, timestamp2) and case_id1 == case_id2:
                      group_key = f"{case_id1}-{timestamp1}"
                      similar_groups.setdefault(group_key, []).append((case_id1, event_name1, timestamp1))
                      break

# Print groups of events with similar timestamps and case IDs
for group_key, events in similar_groups.items():
  if len(events)>1:
    print(f"Sample group of events {group_key}:")
    for case_id, event_name, timestamp in events:
        print(f"\tCase_ID: {case_id}, Event: {event_name}, Timestamp: {timestamp}")


Sample group of events case_17-1970-01-01T06:30:00+05:30:
	Case_ID: case_17, Event: Activity E, Timestamp: 1970-01-01T06:30:00+05:30
	Case_ID: case_17, Event: Activity D, Timestamp: 1970-01-01T06:30:00+05:30
	Case_ID: case_17, Event: Activity C, Timestamp: 1970-01-01T06:30:00+05:30
	Case_ID: case_17, Event: Activity F, Timestamp: 1970-01-01T06:30:00+05:30
Sample group of events case_17-1970-01-01T08:30:00+05:30:
	Case_ID: case_17, Event: Activity G, Timestamp: 1970-01-01T08:30:00+05:30
	Case_ID: case_17, Event: Activity H, Timestamp: 1970-01-01T08:30:00+05:30
	Case_ID: case_17, Event: Activity I, Timestamp: 1970-01-01T08:30:00+05:30
	Case_ID: case_17, Event: Activity J, Timestamp: 1970-01-01T08:30:00+05:30
Sample group of events case_26-1970-01-01T06:30:00+05:30:
	Case_ID: case_26, Event: Activity C, Timestamp: 1970-01-01T06:30:00+05:30
	Case_ID: case_26, Event: Activity F, Timestamp: 1970-01-01T06:30:00+05:30
	Case_ID: case_26, Event: Activity D, Timestamp: 1970-01-01T06:30:00+05:30
	

# LLAMA

In [6]:
!export REPLICATE_API_TOKEN=r8_TUnuAa6pVP5xwFlyTOEX2N9akBdi49y2G2k7S

In [1]:
import os
os.environ["REPLICATE_API_TOKEN"] = "r8_TUnuAa6pVP5xwFlyTOEX2N9akBdi49y2G2k7S"

In [2]:
!pip install replicate

In [23]:
input = {
    "top_p": 1,
    "prompt": "Checking blood, checking pressure, checking oxygen level",
    "temperature": 0.5,
    "system_prompt": "Given the following events, can they be included in a single form? Can they be the fields in a single type of form? Are these events naturally fit together in a cohesive process or task. Give the answer only in a yes or no format without any additional words.",
    "max_new_tokens": 500
}

for event in replicate.stream(
    "meta/llama-2-70b-chat",
    input=input
):
    print(event, end="")




















Yes

In [6]:
import replicate

input = {
    "top_p": 1,
    "prompt": "Checking blood, checking pressure, checking oxygen level",
    "temperature": 0.5,
    "system_prompt": "Aggregate the given events into a single event which describes and includes all the given events altogether. Give only the answer to the asked question without any initial words such as 'sure, here's your answer'etc. Answer should be correct, relatable, short, descriptiv. Word count of the answer should be less than 5 words.",
    "max_new_tokens": 500
}

for event in replicate.stream(
    "meta/llama-2-70b-chat",
    input=input
):
    print(event, end="")
#=> "Sure, here's a poem about open source machine learning i...



"Medical check-up"

In [ ]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=d6ec61f803fafb355e3f19592fd7b04a3fbc47b4bd70b34b0cd0b718e4dce793
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c

In [ ]:
from googletrans import Translator

# Initialize the translator
translator = Translator()

# Function to translate a single value
def translate(value):
    translation = translator.translate(value, src='nl', dest='en')
    return translation.text

# Translate values in the 'concept:name' column
df['concept:name'][:5] = df['concept:name'][:5].apply(translate)

# Display the translated DataFrame
print(df)

                              org:group  Number of executions  \
0                          Radiotherapy                     1   
1                          Radiotherapy                     1   
2                          Nursing ward                     1   
3       Obstetrics & Gynaecology clinic                     1   
4                          Nursing ward                     1   
...                                 ...                   ...   
150286   General Lab Clinical Chemistry                     1   
150287   General Lab Clinical Chemistry                     1   
150288   General Lab Clinical Chemistry                     1   
150289   General Lab Clinical Chemistry                     1   
150290   General Lab Clinical Chemistry                     1   

        Specialism code                           concept:name Producer code  \
0                    61            1st consultation outpatient          SRTH   
1                    61        Administrative rate - First 

<ipython-input-32-b4c2d84edb0c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['concept:name'][:5] = df['concept:name'][:5].apply(translate)
